do pliku `build.gradle(Module)`

```kotlin
dependencies {

    implementation("androidx.lifecycle:lifecycle-viewmodel-compose:2.9.2")

    ...
}
```

```kotlin
class MainActivity : ComponentActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        enableEdgeToEdge()
        setContent {
            WithContextBasicExampleTheme {
                StreamerDashboardScreen()
            }
        }
    }
}

// =================================================================================
// --- ViewModel Streamera ---
// =================================================================================

data class StreamerUiState(
    val isLoading: Boolean = false,
    val status: String = "Na żywo: Interakcja z czatem"
)

class StreamerViewModel : ViewModel() {
    private val _uiState = MutableStateFlow(StreamerUiState())
    val uiState: StateFlow<StreamerUiState> = _uiState.asStateFlow()

    fun startVideoRender() {
        viewModelScope.launch { // -> startuje na Dispatchers.Main
            _uiState.update { it.copy(isLoading = true, status = "Wysyłam pliki do serwerowni...") }

            // Przenosimy blok obliczeniowy na dedykowany wątek za pomocą withContext
            val result = withContext(Dispatchers.Default) {
                // Ten blok wykonuje się w tle na wątku z puli "Default"
                // UI jest w tym czasie w pełni responsywne
                simulateHeavyCpuRender()
            }

            _uiState.update { it.copy(isLoading = false, status = result) }
        }
    }

    private suspend fun simulateHeavyCpuRender(): String {
        var progress = 0
        for (i in 1..100) {
            Thread.sleep(30)
            progress = i
        }
        return "Renderowanie 4K zakończone! ($progress%)"
    }
}

// =================================================================================
// --- UI (Widok) ---
// =================================================================================

@OptIn(ExperimentalMaterial3Api::class)
@Composable
fun StreamerDashboardScreen(viewModel: StreamerViewModel = viewModel()) {
    val uiState by viewModel.uiState.collectAsStateWithLifecycle()

    Scaffold(topBar = { TopAppBar(title = { Text("Panel Streamera") }) }) { padding ->
        Column(
            modifier = Modifier
                .fillMaxSize()
                .padding(padding)
                .padding(16.dp),
            horizontalAlignment = Alignment.CenterHorizontally,
            verticalArrangement = Arrangement.Center
        ) {
            Text("Status Transmisji", style = MaterialTheme.typography.headlineSmall)
            Text(
                text = uiState.status,
                style = MaterialTheme.typography.bodyLarge,
                textAlign = TextAlign.Center,
                modifier = Modifier.padding(16.dp)
            )

            if (uiState.isLoading) {
                CircularProgressIndicator(modifier = Modifier.size(48.dp))
            }

            Spacer(Modifier.height(50.dp))

            Button(
                onClick = { viewModel.startVideoRender() },
                enabled = !uiState.isLoading,
                modifier = Modifier.fillMaxWidth().height(50.dp)
            ) {
                Text("Rozpocznij renderowanie wideo (Praca w tle)")
            }

            Spacer(Modifier.height(50.dp))
            var sliderPosition by remember { mutableFloatStateOf(0f) }
            Slider(
                value = sliderPosition,
                onValueChange = { sliderPosition = it }
            )
            Text("Przesuń, aby sprawdzić responsywność UI", style = MaterialTheme.typography.labelSmall)
        }
    }
}
```